In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

'sh' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import os
get_ipython().system = os.system

In [3]:
!ollama serve &

0

In [4]:
!ollama run gemma3

1

In [5]:
!pip install langchain-community

0

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

In [18]:

system_prompt = f"""
You are a customer churn prediction assistant.
Extract customer features in the exact format:

customerID: string, use "unknown" if not provided
gender: "Male" or "Female"
Senior_Citizen : 0 or 1
Is_Married: "Yes" or "No"
Dependents: "Yes" or "No"
tenure: integer
Phone_Service: "Yes" or "No"
Dual: "Yes" or "No"
Internet_Service: "DSL", "Fiber optic", or "No"
Online_Security: "Yes" or "No"
Online_Backup: "Yes" or "No"
Device_Protection: "Yes" or "No"
Tech_Support: "Yes" or "No"
Streaming_TV: "Yes" or "No"
Streaming_Movies: "Yes" or "No"
Contract: "Month-to-month", "One year", "Two year"
Paperless_Billing: "Yes" or "No"
Payment_Method: "Electronic check", "Mailed check", "Bank transfer (automatic)", "Credit card (automatic)"
Monthly_Charges: float
Total_Charges: float

Return ONLY a Python dictionary with these exact keys:
customerID, gender, Senior_Citizen , Is_Married, Dependents,
tenure, Phone_Service, Dual, Internet_Service, Online_Security,
Online_Backup, Device_Protection, Tech_Support, Streaming_TV,
Streaming_Movies, Contract, Paperless_Billing, Payment_Method,
Monthly_Charges, Total_Charges

- Fill missing features with None (or leave missing if you want NaN) except customerID fill it with unKnown .
- Do NOT add explanations or text outside the dictionary. 
- Use this format: 
{{{{"customerID": "unknown", "gender": "Female", "Senior_Citizen ": 0, ...}}}}

"""



In [28]:
human_prompt= "I have a female customer  , 45 years old, married, no dependents, no internet Service, has phone service, monthly charges 75, total charges 1200 , Paperless_Billing yes, contract month to month, payment method is electronic check, tenure 16 months, no online security, no online backup, no device protection, no tech support, no streaming TV, no streaming movies. What are her features?"


In [36]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{customer_paragraph}")
])

# --- Format the prompt with the human input ---
final_prompt = chat_template.format_messages(customer_paragraph=human_prompt)
llm = ChatOllama(model="gemma3")
response = llm(final_prompt)

print(response.content)

```python
{
"customerID": "unknown",
"gender": "Female",
"Senior_Citizen": 0,
"Is_Married": "Yes",
"Dependents": "No",
"tenure": 16,
"Phone_Service": "Yes",
"Dual": "No",
"Internet_Service": "No",
"Online_Security": "No",
"Online_Backup": "No",
"Device_Protection": "No",
"Tech_Support": "No",
"Streaming_TV": "No",
"Streaming_Movies": "No",
"Contract": "Month-to-month",
"Paperless_Billing": "Yes",
"Payment_Method": "Electronic check",
"Monthly_Charges": 75.0,
"Total_Charges": 1200.0
}
```


In [10]:
print(type(response.content))


<class 'str'>


In [17]:
print(response.content[:500])  # see the first 500 characters
  # check if it’s empty


```python
{
    "customerID": "unknown",
    "gender": "Female",
    "Senior_Citizen": 0,
    "Is_Married": "Yes",
    "Dependents": "No",
    "tenure": None,
    "Phone_Service": "Yes",
    "Dual": None,
    "Internet_Service": "No",
    "Online_Security": None,
    "Online_Backup": None,
    "Device_Protection": None,
    "Tech_Support": None,
    "Streaming_TV": None,
    "Streaming_Movies": None,
    "Contract": None,
    "Paperless_Billing": None,
    "Payment_Method": None,
    "Monthly_Ch


In [37]:
import json
import re
import ast

def debug_and_convert_llm_output(llm_string):
    """
    Debug and convert LLM output to dictionary with detailed error handling
    """
    
    print("="*50)
    print("DEBUGGING LLM OUTPUT")
    print("="*50)
    
    # Step 1: Check what we received
    print(f"Raw output type: {type(llm_string)}")
    print(f"Raw output length: {len(llm_string) if llm_string else 0}")
    print(f"Raw output (first 200 chars): {repr(llm_string[:200])}")
    
    if not llm_string or len(llm_string.strip()) == 0:
        print("ERROR: Empty or None input!")
        return None
    
    # Step 2: Clean the string
    cleaned = llm_string.strip()
    
    # Step 3: Try to extract JSON from the response
    # Look for JSON-like patterns (starts with { and ends with })
    json_match = re.search(r'\{.*\}', cleaned, re.DOTALL)
    
    if json_match:
        json_candidate = json_match.group(0)
        print(f"\nFound JSON-like content: {json_candidate[:100]}...")
    else:
        print(f"\nNo JSON pattern found. Full content:\n{cleaned}")
        # Try to find any dictionary-like content
        dict_patterns = [
            r'```python\s*(\{.*?\})\s*```',  # Python code blocks
            r'```json\s*(\{.*?\})\s*```',    # JSON code blocks  
            r'```\s*(\{.*?\})\s*```',        # Generic code blocks
        ]
        
        for pattern in dict_patterns:
            match = re.search(pattern, cleaned, re.DOTALL)
            if match:
                json_candidate = match.group(1)
                print(f"Found content in code block: {json_candidate[:100]}...")
                break
        else:
            return None
    
    # Step 4: Try different conversion methods
    conversion_methods = [
        ("JSON with None->null replacement", lambda x: json.loads(x.replace('None', 'null').replace('True', 'true').replace('False', 'false'))),
        ("AST literal eval", ast.literal_eval),
        ("JSON direct", json.loads),
        ("Eval (unsafe)", eval),
    ]
    
    for method_name, method_func in conversion_methods:
        try:
            print(f"\nTrying method: {method_name}")
            result = method_func(json_candidate)
            print(f"✅ SUCCESS with {method_name}")
            print(f"Result type: {type(result)}")
            print(f"Result keys: {list(result.keys()) if isinstance(result, dict) else 'Not a dict'}")
            return result
        except Exception as e:
            print(f"❌ Failed with {method_name}: {str(e)}")
    
    return None

def robust_llm_to_dict(llm_string):
    """
    More robust conversion function
    """
    if not llm_string:
        return None
    
    # Clean the input
    cleaned = str(llm_string).strip()
    
    # Extract JSON-like content
    patterns = [
        r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}',  # Simple nested braces
        r'\{.*\}',  # Any content between braces
    ]
    
    json_content = None
    for pattern in patterns:
        matches = re.findall(pattern, cleaned, re.DOTALL)
        if matches:
            # Take the longest match (likely the most complete)
            json_content = max(matches, key=len)
            break
    
    if not json_content:
        return None
    
    # Try conversion methods in order of preference
    methods = [
        lambda x: json.loads(re.sub(r'\b(None|True|False)\b', 
                                   lambda m: {'None': 'null', 'True': 'true', 'False': 'false'}[m.group()], x)),
        ast.literal_eval,
    ]
    
    for method in methods:
        try:
            result = method(json_content)
            if isinstance(result, dict):
                return result
        except:
            continue
    
    return None

# Test with your actual response
# Replace this with your actual response.content
response = response.content

print("TESTING WITH SAMPLE DATA:")
result = debug_and_convert_llm_output(response)
if result:
    print(f"\n✅ Final result: {result}")
else:
    print("\n❌ Could not convert to dictionary")


TESTING WITH SAMPLE DATA:
DEBUGGING LLM OUTPUT
Raw output type: <class 'str'>
Raw output length: 492
Raw output (first 200 chars): '```python\n{\n"customerID": "unknown",\n"gender": "Female",\n"Senior_Citizen": 0,\n"Is_Married": "Yes",\n"Dependents": "No",\n"tenure": 16,\n"Phone_Service": "Yes",\n"Dual": "No",\n"Internet_Service": "No",\n"On'

Found JSON-like content: {
"customerID": "unknown",
"gender": "Female",
"Senior_Citizen": 0,
"Is_Married": "Yes",
"Dependents...

Trying method: JSON with None->null replacement
✅ SUCCESS with JSON with None->null replacement
Result type: <class 'dict'>
Result keys: ['customerID', 'gender', 'Senior_Citizen', 'Is_Married', 'Dependents', 'tenure', 'Phone_Service', 'Dual', 'Internet_Service', 'Online_Security', 'Online_Backup', 'Device_Protection', 'Tech_Support', 'Streaming_TV', 'Streaming_Movies', 'Contract', 'Paperless_Billing', 'Payment_Method', 'Monthly_Charges', 'Total_Charges']

✅ Final result: {'customerID': 'unknown', 'gender': 'Female', 'S

In [48]:
import sys
import os

# Add the source folder to sys.path
sys.path.append(os.path.abspath('../src'))

# Now you can import functions from model.py
from model import classify_customer

final_result = classify_customer(result)


In [47]:
import importlib
import model
importlib.reload(model)

from model import classify_customer


In [49]:
print(final_result)

{'Customer_ID': 'unknown', 'prediction': 0, 'Reasons': ['multi__Internet_Service_No ↓ (impact=-4.096)', 'bin__Phone_Service ↑ (impact=1.245)', 'num__Monthly_Charges ↓ (impact=-0.795)', 'bin__Streaming_Movies ↑ (impact=0.599)', 'bin__Streaming_TV ↑ (impact=0.597)']}


In [56]:
def build_marketing_prompt(output: dict) -> str:
    pred = "will churn" if output["prediction"] == 1 else "will stay"
  
    
    reasons = "\n".join([f"- {r}" for r in output["Reasons"]])
    
    return f"""
You are a customer success advisor. 
The model predicts that this customer {pred} .

Here are the technical factors:
{reasons}

Now explain to a marketing team in very natural business language:
- Why the customer is likely to {pred}
- What the key reasons are, phrased in everyday terms
- Keep the explanation clear, short, and actionable
"""


In [57]:
system_prompt_response = (
    "You are a helpful assistant that explains customer churn predictions "
    "in plain, natural language for a marketing team."
)

# Use the marketing-friendly prompt builder instead of the old one
human_prompt_response = build_marketing_prompt(final_result)

chat_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt_response),
    ("human", "{customer_paragraph}")
])

final_reason_prompt = chat_template.format_messages(
    customer_paragraph=human_prompt_response
)

llm = ChatOllama(model="gemma3")
reasons = llm(final_reason_prompt)

print(reasons.content)


Okay team, let’s talk about this customer – we’ve got a good prediction they’re going to stick around. 

Basically, the model is saying they’re pretty happy with us right now. The biggest factors keeping them loyal are that they’re using our internet service and that their monthly bill is relatively low. 

However, we should be aware that they’re actively using our streaming TV service, which is a positive sign, but not a huge driver of loyalty.  It's worth noting they're also using our phone service – that’s holding them steady. 

**Actionable takeaway:** Let’s focus on reinforcing the value of our internet service – maybe a quick check-in to make sure they’re getting the most out of it. We can also subtly highlight the convenience of our streaming options, but let’s not overemphasize that compared to their internet usage. 


---

Do you want me to tailor this explanation for a specific channel (e.g., email, phone call)?
